In [ ]:
# Data Analysis - 
# Version 1
## 1/14/25
## 15:30

In [25]:
# Importing Modules
import sys
import os

# Add the root directory to sys.path
sys.path.append(os.path.abspath(".."))

# Now you can use absolute imports
from d497_helpers import folder_manager as fm, archive_module, checkpoint_helper as CheckPoint, database_helper as db_tool, config, import_dataframe

#### #### #### #### 


import itertools

# Pandas
import pandas as pd

# Numpy
import numpy as np

# RegularExpression for string matching
import re

# GLOB 
import glob 

# Pathlib 
from pathlib import Path
    
# CSV
import csv

# RapidFuzzr
from rapidfuzz import process

In [2]:
import_cdc_data_main_csv, import_cdc_data_main_pickle = import_dataframe.import_dataframe("cdc_data", "cleaned", "cdc_data_main_df")

In [3]:
import_cdc_data_main_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204528 entries, 0 to 204527
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   year_code    204528 non-null  object
 1   month_code   204528 non-null  object
 2   state_code   204528 non-null  object
 3   county_code  204528 non-null  object
 4   births       204528 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 7.8+ MB


In [4]:
import_cdc_data_main_pickle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204528 entries, 0 to 204527
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   year_code    204528 non-null  object
 1   month_code   204528 non-null  object
 2   state_code   204528 non-null  object
 3   county_code  204528 non-null  object
 4   births       204528 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 7.8+ MB


In [5]:
cdc_data_main_df = import_cdc_data_main_pickle.copy()

In [7]:
del import_cdc_data_main_csv
del import_cdc_data_main_pickle

In [8]:
cdc_data_main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204528 entries, 0 to 204527
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   year_code    204528 non-null  object
 1   month_code   204528 non-null  object
 2   state_code   204528 non-null  object
 3   county_code  204528 non-null  object
 4   births       204528 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 7.8+ MB


In [10]:
cdc_data_main_df.head(5)

,year_code,month_code,state_code,county_code,births
0,1995,1,01,01015,127
1,1995,2,01,01015,127
2,1995,3,01,01015,127
3,1995,4,01,01015,127
4,1995,5,01,01015,127


In [43]:
cdc_data_main_df.rename(columns={'county_code':'fips_five', 'state_code':'state_fipcode'}, inplace=True )

In [44]:
cdc_data_main_df['county_fipcode'] = "C" + cdc_data_main_df['fips_five'].str.slice(-3)

In [46]:
cdc_data_main_df.insert(3, 'county_fipcode', cdc_data_main_df.pop('county_fipcode'))

In [47]:
cdc_data_main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204528 entries, 0 to 204527
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   year_code       204528 non-null  object
 1   month_code      204528 non-null  object
 2   state_fipcode   204528 non-null  object
 3   county_fipcode  204528 non-null  object
 4   fips_five       204528 non-null  object
 5   births          204528 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 9.4+ MB


In [48]:
cdc_data_main_df.sample(5)

,year_code,month_code,state_fipcode,county_fipcode,fips_five,births
103206,2010,7,49,C999,49999,1064
180186,2020,10,18,C089,18089,452
9978,1996,7,37,C119,37119,784
19126,1998,11,09,C009,09009,880
12929,1997,6,08,C069,08069,234


In [11]:
sql_query = """

    SELECT 
        state_fipcode,
        county_fipcode,
        fips_five
    FROM
        fips_data
"""

In [13]:
result = db_tool.execute_query(sql_query)

2025-01-14 16:21:22,507 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-14 16:21:22,508 INFO sqlalchemy.engine.Engine 

    SELECT 
        state_fipcode,
        county_fipcode,
        fips_five
    FROM
        fips_data

2025-01-14 16:21:22,509 INFO sqlalchemy.engine.Engine [cached since 172.7s ago] ()
2025-01-14 16:21:22,566 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
fips_data = pd.DataFrame.from_records(result)

In [18]:
fips_data.columns = ['state_fipcode', 'county_fipcode', 'fips_five']

In [20]:
fips_data.sample()

,state_fipcode,county_fipcode,fips_five
18449,29,C005,29005


In [22]:
fips_five_list = fips_data['fips_five'].unique()

In [23]:
fips_five_list

array(['01001', '01003', '01005', ..., '56041', '56043', '56045'],
      shape=(3145,), dtype=object)

In [89]:
# Define the range of years and months
years = list(range(1995, 2024)) # 1994 to 2023 inclusive
months = list(range(1, 13))      # 1 to 12 inclusive

In [90]:
all_combinations = pd.DataFrame(
    itertools.product(years, months, fips_five_list),
    columns=['year_code', 'month_code', 'fips_five']
)

In [91]:
all_combinations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094460 entries, 0 to 1094459
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   year_code   1094460 non-null  int64 
 1   month_code  1094460 non-null  int64 
 2   fips_five   1094460 non-null  object
dtypes: int64(2), object(1)
memory usage: 25.1+ MB


In [92]:
all_combinations.head(25)

,year_code,month_code,fips_five
0,1995,1,01001
1,1995,1,01003
2,1995,1,01005
3,1995,1,01007
4,1995,1,01009
5,1995,1,01011
6,1995,1,01013
7,1995,1,01015
8,1995,1,01017
9,1995,1,01019


In [93]:
all_combinations['state_fipcode'] = all_combinations['fips_five'].str.slice(0, 2)

In [94]:
all_combinations['county_fipcode'] = "C" + all_combinations['fips_five'].str.slice(-3)

In [95]:
all_combinations.sample(5)

,year_code,month_code,fips_five,state_fipcode,county_fipcode
533428,2009,2,37065,37,C065
817955,2016,9,08027,08,C027
666,1995,1,17147,17,C147
724049,2014,3,18007,18,C007
716686,2013,12,48495,48,C495


In [96]:
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### ####                                                   #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### ####                   CHECK POINT                     #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### ####                                                   #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 


CheckPoint.create_checkpoint("results", "all_combinations_initial", all_combinations)


#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### ####                                                   #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### ####                   CHECK POINT                     #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### ####                                                   #### #### #### #### #### #### #### #### #### #### #### #### 
#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

In [97]:
all_combinations.head(10)

,year_code,month_code,fips_five,state_fipcode,county_fipcode
0,1995,1,01001,01,C001
1,1995,1,01003,01,C003
2,1995,1,01005,01,C005
3,1995,1,01007,01,C007
4,1995,1,01009,01,C009
5,1995,1,01011,01,C011
6,1995,1,01013,01,C013
7,1995,1,01015,01,C015
8,1995,1,01017,01,C017
9,1995,1,01019,01,C019


In [98]:
all_combinations['year_code'] = all_combinations['year_code'].astype("int")
all_combinations['month_code'] = all_combinations['month_code'].astype("int")

In [99]:
cdc_data_main_df['year_code'] = cdc_data_main_df['year_code'].astype("int")
cdc_data_main_df['month_code'] = cdc_data_main_df['month_code'].astype("int")

In [100]:
all_combinations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094460 entries, 0 to 1094459
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   year_code       1094460 non-null  int64 
 1   month_code      1094460 non-null  int64 
 2   fips_five       1094460 non-null  object
 3   state_fipcode   1094460 non-null  object
 4   county_fipcode  1094460 non-null  object
dtypes: int64(2), object(3)
memory usage: 41.8+ MB


In [101]:
merging_df = all_combinations.merge(cdc_data_main_df, on=['year_code','month_code', 'fips_five'], how="left", indicator=True)

In [102]:
merging_df.head(25)

,year_code,month_code,fips_five,state_fipcode_x,county_fipcode_x,state_fipcode_y,county_fipcode_y,births,_merge
0,1995,1,01001,01,C001,NaN,NaN,NaN,left_only
1,1995,1,01003,01,C003,NaN,NaN,NaN,left_only
2,1995,1,01005,01,C005,NaN,NaN,NaN,left_only
3,1995,1,01007,01,C007,NaN,NaN,NaN,left_only
4,1995,1,01009,01,C009,NaN,NaN,NaN,left_only
5,1995,1,01011,01,C011,NaN,NaN,NaN,left_only
6,1995,1,01013,01,C013,NaN,NaN,NaN,left_only
7,1995,1,01015,01,C015,01,C015,127.0,both
8,1995,1,01017,01,C017,NaN,NaN,NaN,left_only
9,1995,1,01019,01,C019,NaN,NaN,NaN,left_only


In [103]:
merging_df.loc[merging_df['births'].notnull()]

,year_code,month_code,fips_five,state_fipcode_x,county_fipcode_x,state_fipcode_y,county_fipcode_y,births,_merge
7,1995,1,01015,01,C015,01,C015,127.0,both
35,1995,1,01073,01,C073,01,C073,781.0,both
43,1995,1,01089,01,C089,01,C089,321.0,both
47,1995,1,01097,01,C097,01,C097,508.0,both
49,1995,1,01101,01,C101,01,C101,293.0,both
...,...,...,...,...,...,...,...,...,...
1094424,2023,12,55117,55,C117,55,C117,92.0,both
1094429,2023,12,55127,55,C127,55,C127,68.0,both
1094431,2023,12,55131,55,C131,55,C131,102.0,both
1094432,2023,12,55133,55,C133,55,C133,282.0,both


In [110]:
merging_df['_merge'].unique()

['left_only', 'both']
Categories (3, object): ['left_only', 'right_only', 'both']

In [104]:
missing_data_df = merging_df[merging_df['_merge'] == "left_only"]

In [105]:
missing_data_df

,year_code,month_code,fips_five,state_fipcode_x,county_fipcode_x,state_fipcode_y,county_fipcode_y,births,_merge
0,1995,1,01001,01,C001,NaN,NaN,NaN,left_only
1,1995,1,01003,01,C003,NaN,NaN,NaN,left_only
2,1995,1,01005,01,C005,NaN,NaN,NaN,left_only
3,1995,1,01007,01,C007,NaN,NaN,NaN,left_only
4,1995,1,01009,01,C009,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...
1094455,2023,12,56037,56,C037,NaN,NaN,NaN,left_only
1094456,2023,12,56039,56,C039,NaN,NaN,NaN,left_only
1094457,2023,12,56041,56,C041,NaN,NaN,NaN,left_only
1094458,2023,12,56043,56,C043,NaN,NaN,NaN,left_only


In [108]:
cdc_only_data_df = merging_df[merging_df['_merge'] == "right_only"]

In [109]:
cdc_only_data_df

,year_code,month_code,fips_five,state_fipcode_x,county_fipcode_x,state_fipcode_y,county_fipcode_y,births,_merge


In [106]:
print(cdc_data_main_df.dtypes)

year_code          int64
month_code         int64
state_fipcode     object
county_fipcode    object
fips_five         object
births             int64
dtype: object


In [107]:
test_merge = all_combinations.merge(
    cdc_data_main_df, 
    on=['year_code', 'month_code', 'state_fipcode', 'county_fipcode'], 
    how='inner'
)
print(f"Number of matching rows: {len(test_merge)}")

Number of matching rows: 187524


In [113]:
print(all_combinations.shape[0] - missing_data_df.shape[0])

187524


In [121]:
cdc_data_main_df.shape[0]

204528

In [119]:
right_merge_df = cdc_data_main_df.merge(all_combinations, on=['year_code','month_code', 'state_fipcode', 'county_fipcode', 'fips_five'], how="left", indicator=True)

In [120]:
right_merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204528 entries, 0 to 204527
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   year_code       204528 non-null  int64   
 1   month_code      204528 non-null  int64   
 2   state_fipcode   204528 non-null  object  
 3   county_fipcode  204528 non-null  object  
 4   fips_five       204528 non-null  object  
 5   births          204528 non-null  int64   
 6   _merge          204528 non-null  category
dtypes: category(1), int64(3), object(3)
memory usage: 9.6+ MB


In [123]:
non_matched_cdc_data_df = right_merge_df[right_merge_df['_merge'] == "left_only"]

In [124]:
non_matched_cdc_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17004 entries, 96 to 204527
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   year_code       17004 non-null  int64   
 1   month_code      17004 non-null  int64   
 2   state_fipcode   17004 non-null  object  
 3   county_fipcode  17004 non-null  object  
 4   fips_five       17004 non-null  object  
 5   births          17004 non-null  int64   
 6   _merge          17004 non-null  category
dtypes: category(1), int64(3), object(3)
memory usage: 946.6+ KB


In [125]:
non_matched_cdc_data_df.sample(30)

,year_code,month_code,state_fipcode,county_fipcode,fips_five,births,_merge
114899,2012,2,30,C999,30999,785,left_only
198955,2023,4,09,C999,09999,0,left_only
204229,2023,12,33,C999,33999,214,left_only
149174,2016,8,55,C999,55999,1929,left_only
119999,2012,10,36,C999,36999,1652,left_only
70128,2006,2,22,C999,22999,2044,left_only
69918,2006,1,55,C999,55999,2012,left_only
100894,2010,4,13,C999,13999,4882,left_only
130272,2014,2,51,C999,51999,2975,left_only
107602,2011,2,48,C999,48999,4924,left_only


In [126]:
non_matched_cdc_data_df['county_fipcode'].unique()

array(['C999', 'C025'], dtype=object)

In [130]:
non_matched_cdc_data_df['fips_five'].unique()

array(['01999', '02999', '04999', '05999', '06999', '08999', '12025',
       '12999', '13999', '15999', '16999', '17999', '18999', '19999',
       '20999', '21999', '22999', '23999', '24999', '25999', '26999',
       '27999', '28999', '29999', '30999', '31999', '32999', '33999',
       '34999', '35999', '36999', '37999', '38999', '39999', '40999',
       '41999', '42999', '44999', '45999', '46999', '47999', '48999',
       '49999', '50999', '51999', '53999', '54999', '55999', '56999',
       '09999'], dtype=object)

In [132]:
non_matched_cdc_data_df[(non_matched_cdc_data_df['state_fipcode'] == "12")]

,year_code,month_code,state_fipcode,county_fipcode,fips_five,births,_merge
1008,1995,1,12,C025,12025,2676,left_only
1009,1995,2,12,C025,12025,2676,left_only
1010,1995,3,12,C025,12025,2676,left_only
1011,1995,4,12,C025,12025,2675,left_only
1012,1995,5,12,C025,12025,2675,left_only
...,...,...,...,...,...,...,...
201509,2023,8,12,C999,12999,1155,left_only
202138,2023,9,12,C999,12999,1202,left_only
202767,2023,10,12,C999,12999,1229,left_only
203396,2023,11,12,C999,12999,1131,left_only


In [135]:
non_matched_cdc_data_df[(non_matched_cdc_data_df['state_fipcode'] == "12") & (non_matched_cdc_data_df['county_fipcode'] == "C999")]

,year_code,month_code,state_fipcode,county_fipcode,fips_five,births,_merge
1272,1995,1,12,C999,12999,1339,left_only
1273,1995,2,12,C999,12999,1339,left_only
1274,1995,3,12,C999,12999,1339,left_only
1275,1995,4,12,C999,12999,1339,left_only
1276,1995,5,12,C999,12999,1339,left_only
...,...,...,...,...,...,...,...
201509,2023,8,12,C999,12999,1155,left_only
202138,2023,9,12,C999,12999,1202,left_only
202767,2023,10,12,C999,12999,1229,left_only
203396,2023,11,12,C999,12999,1131,left_only


In [136]:
non_matched_cdc_data_df[(non_matched_cdc_data_df['state_fipcode'] == "12") & (non_matched_cdc_data_df['county_fipcode'] == "C025")]

,year_code,month_code,state_fipcode,county_fipcode,fips_five,births,_merge
1008,1995,1,12,C025,12025,2676,left_only
1009,1995,2,12,C025,12025,2676,left_only
1010,1995,3,12,C025,12025,2676,left_only
1011,1995,4,12,C025,12025,2675,left_only
1012,1995,5,12,C025,12025,2675,left_only
...,...,...,...,...,...,...,...
43687,2002,8,12,C025,12025,2677,left_only
43688,2002,9,12,C025,12025,2677,left_only
43689,2002,10,12,C025,12025,2677,left_only
43690,2002,11,12,C025,12025,2677,left_only


In [127]:
ucr_sql_query = """

    SELECT
        *
    FROM
        fips_data
        WHERE county_fipcode == "C999"

"""

In [128]:
Unspecified_county_results = db_tool.execute_query(ucr_sql_query)

2025-01-15 13:12:31,661 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-15 13:12:31,666 INFO sqlalchemy.engine.Engine 

    SELECT
        *
    FROM
        fips_data
        WHERE county_fipcode == "C999"


2025-01-15 13:12:31,667 INFO sqlalchemy.engine.Engine [generated in 0.00206s] ()
2025-01-15 13:12:31,681 INFO sqlalchemy.engine.Engine COMMIT


In [129]:
Unspecified_county_results

[]

In [138]:
non_matched_cdc_data_df[non_matched_cdc_data_df['county_fipcode'] == "C025"]['state_fipcode'].unique()

array(['12'], dtype=object)